In [1]:
document = """To Sherlock Holmes she is always the woman. I have
seldom heard him mention her under any other name. In his eyes she
eclipses and predominates the whole of her sex. It was not that he
felt any emotion akin to love for Irene Adler. All emotions, and that
one particularly, were abhorrent to his cold, precise but admirably
balanced mind. He was, I take it, the most perfect reasoning and
observing machine that the world has seen, but as a lover he would
have placed himself in a false position. He never spoke of the softer
passions, save with a gibe and a sneer. They were admirable things for
the observer-excellent for drawing the veil from men’s motives and
actions. But for the trained reasoner to admit such intrusions into
his own delicate and finely adjusted temperament was to introduce a
distracting factor which might throw a doubt upon all his mental
results. Grit in a sensitive instrument, or a crack in one of his own
high-power lenses, would not be more disturbing than a strong emotion
in a nature such as his. And yet there was but one woman to him, and
that woman was the late Irene Adler, of dubious and questionable
memory.
"""

In [2]:
from gensim.summarization import summarize
print summarize(document)

felt any emotion akin to love for Irene Adler.
one particularly, were abhorrent to his cold, precise but admirably
He was, I take it, the most perfect reasoning and
But for the trained reasoner to admit such intrusions into
that woman was the late Irene Adler, of dubious and questionable


In [3]:
import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer

In [4]:
sentence_tokenizer = PunktSentenceTokenizer()
sentences_original = sentence_tokenizer.tokenize(" ".join(document.strip().split("\n")))

In [5]:
from nltk.corpus import stopwords

In [6]:
stops = set(stopwords.words("english"))

In [7]:
document = document.lower().split()
document = [w for w in document if not w in stops]

In [8]:
document = " ".join(document)
document

'sherlock holmes always woman. seldom heard mention name. eyes eclipses predominates whole sex. felt emotion akin love irene adler. emotions, one particularly, abhorrent cold, precise admirably balanced mind. was, take it, perfect reasoning observing machine world seen, lover would placed false position. never spoke softer passions, save gibe sneer. admirable things observer-excellent drawing veil men\xe2\x80\x99s motives actions. trained reasoner admit intrusions delicate finely adjusted temperament introduce distracting factor might throw doubt upon mental results. grit sensitive instrument, crack one high-power lenses, would disturbing strong emotion nature his. yet one woman him, woman late irene adler, dubious questionable memory.'

In [9]:
document = " ".join(document.strip().split("\n"))

sentences = sentence_tokenizer.tokenize(document)

In [10]:
document

'sherlock holmes always woman. seldom heard mention name. eyes eclipses predominates whole sex. felt emotion akin love irene adler. emotions, one particularly, abhorrent cold, precise admirably balanced mind. was, take it, perfect reasoning observing machine world seen, lover would placed false position. never spoke softer passions, save gibe sneer. admirable things observer-excellent drawing veil men\xe2\x80\x99s motives actions. trained reasoner admit intrusions delicate finely adjusted temperament introduce distracting factor might throw doubt upon mental results. grit sensitive instrument, crack one high-power lenses, would disturbing strong emotion nature his. yet one woman him, woman late irene adler, dubious questionable memory.'

In [11]:
import re

for i in xrange(0, len(sentences)) :
    sentences[i] = re.sub("[^a-zA-Z]", " ", sentences[i])
    print sentences[i]

sherlock holmes always woman 
seldom heard mention name 
eyes eclipses predominates whole sex 
felt emotion akin love irene adler 
emotions  one particularly  abhorrent cold  precise admirably balanced mind 
was  take it  perfect reasoning observing machine world seen  lover would placed false position 
never spoke softer passions  save gibe sneer 
admirable things observer excellent drawing veil men   s motives actions 
trained reasoner admit intrusions delicate finely adjusted temperament introduce distracting factor might throw doubt upon mental results 
grit sensitive instrument  crack one high power lenses  would disturbing strong emotion nature his 
yet one woman him  woman late irene adler  dubious questionable memory 


In [12]:
from gensim.models import Word2Vec
model = Word2Vec.load("~/Desktop/PClub_DTC/NLP_Learning/Kaggle_BagOfWords/300features_40minwords_10context")

In [13]:
import numpy as np

In [14]:
index2word_set = set(model.wv.vocab)

In [15]:
sentence_matrix = []
sentence_length = []

num_features = 300

for sentence in sentences :
    num_words = 0
    words = sentence.split()
    matrix = np.empty(shape=(len(words), 300), dtype=float)
    for word in words : 
        if word in index2word_set :
            matrix[num_words] = model[word]
            num_words = num_words + 1

    extra_rows = len(words) - num_words
    
    for i in xrange(0, extra_rows) :
        matrix = np.delete(matrix, -1, 0)
        
    sentence_matrix.append(matrix)
    sentence_length.append(num_words)

In [16]:
similarity_matrix = np.identity(len(sentence_matrix))

In [17]:
for i in xrange(0, len(sentence_matrix)) :
    for j in xrange(i+1, len(sentence_matrix)) :
        cum_sum = (np.dot(sentence_matrix[0], sentence_matrix[1].T)).sum(axis=0).sum()
        cum_sum = (2*cum_sum)/(sentence_length[i] + sentence_length[j])
        similarity_matrix[i][j] = cum_sum
        similarity_matrix[j][i] = cum_sum

In [18]:
import networkx as nx
nx_graph = nx.from_numpy_matrix(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [19]:
sentences_original

['To Sherlock Holmes she is always the woman.',
 'I have seldom heard him mention her under any other name.',
 'In his eyes she eclipses and predominates the whole of her sex.',
 'It was not that he felt any emotion akin to love for Irene Adler.',
 'All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind.',
 'He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position.',
 'He never spoke of the softer passions, save with a gibe and a sneer.',
 'They were admirable things for the observer-excellent for drawing the veil from men\xe2\x80\x99s motives and actions.',
 'But for the trained reasoner to admit such intrusions into his own delicate and finely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results.',
 'Grit in a sensitive instrument, or a crack in one of his own high-power lenses

In [20]:
ranked = sorted(((scores[i],i) for (i, s) in enumerate(sentences_original)), reverse=True)

In [21]:
summary_length = len(sentences)/3

for i in xrange(0, summary_length) :
    print sentences_original[ranked[i][1]]

In his eyes she eclipses and predominates the whole of her sex.
He never spoke of the softer passions, save with a gibe and a sneer.
I have seldom heard him mention her under any other name.
